# SEGMENTING AND CLUSTERING NEIGHBORHOODS IN CITY OF TORONTO, CANADA

### Scraping data from wikipedia page :

In [1]:
import bs4
import requests

import numpy as np
import pandas as pd

import folium

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
soup = bs4.BeautifulSoup(res.text,'lxml')

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                     'EtobicokeNorthwest':'Etobicoke Northwest',
                                     'East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [4]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
df.shape

(103, 3)

### Extracting location coordinates of the neighborhoods from csv file :

In [6]:
csv_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
lat_lon = pd.read_csv(csv_path)
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging lat_lon df with neighborhoods df :

In [7]:
lat_lon.rename(columns={'Postal Code':'PostalCode'},inplace=True)
toronto_df = pd.merge(df,lat_lon,on='PostalCode')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Visualizing the toronto_df using folium :
### note : toronto city coordinates : [43.651070, -79.347015]

In [8]:
toronto_map = folium.Map(location=[43.651070, -79.347015],zoom_start = 11)

for lat,lng,borough,neighborhood in zip(toronto_df['Latitude'],toronto_df['Longitude'],toronto_df['Borough'],toronto_df['Neighborhood']):
    label = '{},{}'.format(borough,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        popup=label,
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
    
toronto_map

### Clustering toronto_df using K-Means Clustering :

In [9]:
# Feature set X :
X = toronto_df[['Latitude','Longitude']]

In [10]:
from sklearn.cluster import KMeans

kmeans = KMeans(init='k-means++',n_clusters=5,n_init=12) # considering 5 clusters
kmeans.fit(X)
labels=kmeans.labels_
toronto_df.insert(0,'Cluster labels',labels)
toronto_df.head()

,Cluster labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,1,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [17]:
import matplotlib.cm as cm
import matplotlib.colors as colors

toronto_cluster_map = folium.Map(location=[43.651070, -79.347015],zoom_start=11)

x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat,lng,cluster,neighborhood in zip(toronto_df['Latitude'],toronto_df['Longitude'],toronto_df['Cluster labels'],toronto_df['Neighborhood']):
    label = folium.Popup('Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        popup=label,
        radius=5,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_cluster_map)
    
toronto_cluster_map

In [18]:
# to view the maps, paste the following link in your browser :
# https://nbviewer.jupyter.org/github/Girish-Chandra/Segmenting-Clustering-neighborhoods-of-toronto-city-canada/blob/main/Segmenting%20%26%20Clustering%20neighbourhoods%20in%20city%20of%20Toronto%2C%20Canada.ipynb